In [2]:
import pandas as pd
import numpy as np
import gzip
from Bio import SeqIO   # for reading fasta files
from pyensembl import EnsemblRelease   # to get the gene list
import pybedtools

In [3]:
#!pyensembl install --release 75 --species homo_sapiens

In [4]:
ENSEMBL_RELEASE = 75 #emseble release for GRCh37 (hg19)
DNA_TOPLEVEL_FASTA_PATH = "/home/mrkvrbl/Diplomka/Data/reference/referecnce_genome/hg19_latest.fa"
OUTPUT_FILE_CSV = "/home/mrkvrbl/Diplomka/Data/outputs/random_transcripts.csv"   # where to save them
OUTPUT_FILE_BED = "/home/mrkvrbl/Diplomka/Data/outputs/random_transcripts.bed"
INTRONS = "/home/mrkvrbl/Diplomka/Data/reference/region_types/introns.bed.gz"
EXONS = "/home/mrkvrbl/Diplomka/Data/reference/region_types/exons.bed.gz"
UTR5 = "/home/mrkvrbl/Diplomka/Data/reference/region_types/5utrs.bed.gz"
UTR3 = "/home/mrkvrbl/Diplomka/Data/reference/region_types/3utrs.bed.gz"

# to generate random sequences
N = 90_000    # how many
K = 128     # how long

CHRS = [str(chr) for chr in range(1,23)] + ['X', 'Y', 'MT']

In [5]:
data = EnsemblRelease(ENSEMBL_RELEASE)
human_transcripts = data.transcript_ids()

transcripts_full_info  = [data.transcript_by_id(transcript) for transcript in human_transcripts]

human_transcript_tuples = [(x.transcript_id, x.gene_id, x.biotype, x.contig, x.start, x.end, x.strand, x.coding_sequence) for x in transcripts_full_info if x.contains_start_codon & x.contains_stop_codon]
human_transcript_table = pd.DataFrame.from_records(human_transcript_tuples, columns=["id", "gene_id", "biotype", "chr", "start", "end", "strand", "coding_sequence"])
assert all(human_transcript_table.start <= human_transcript_table.end)

INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/mrkvrbl/.cache/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.cdna.all.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/mrkvrbl/.cache/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.ncrna.fa.gz.pickle


In [6]:
assert ~human_transcript_table.coding_sequence.str.contains('N').any()
human_transcript_table['length'] = human_transcript_table.coding_sequence.apply(len)
human_transcript_table = human_transcript_table[human_transcript_table.biotype == "protein_coding"]
human_transcript_table = human_transcript_table[human_transcript_table.chr.str.fullmatch(pat=r"[\w]{1,2}")]
human_transcript_table

,id,gene_id,biotype,chr,start,end,strand,coding_sequence,length
0,ENST00000000233,ENSG00000004059,protein_coding,7,127228399,127231759,+,ATGGGCCTCACCGTGTCCGCGCTCTTTTCGCGGATCTTCGGGAAGA...,543
1,ENST00000000412,ENSG00000003056,protein_coding,12,9092961,9102551,-,ATGTTCCCTTTCTACAGCTGCTGGAGGACTGGACTGCTACTACTAC...,834
2,ENST00000000442,ENSG00000173153,protein_coding,11,64073050,64084210,+,ATGTCCAGCCAGGTGGTGGGCATTGAGCCTCTCTACATCAAGGCAG...,1272
3,ENST00000001008,ENSG00000004478,protein_coding,12,2904119,2913124,+,ATGACAGCCGAGGAGATGAAGGCGACCGAGAGCGGGGCGCAGTCGG...,1380
4,ENST00000001146,ENSG00000003137,protein_coding,2,72356367,72375167,-,ATGCTCTTTGAGGGCTTGGATCTGGTGTCGGCGCTGGCCACCCTCG...,1539
...,...,...,...,...,...,...,...,...,...
72858,ENST00000610189,ENSG00000198522,protein_coding,2,27851919,27873713,+,ATGGCGGCGTCCGCAGCTGCCGCTGAGCTCCAGGCTTCTGGGGGTC...,1125
72862,ENST00000610205,ENSG00000173464,protein_coding,14,21051054,21058417,-,ATGGAGACCTTTCCTCTGCTGCTGCTCAGCCTGGGCCTGGTTCTTG...,600
72869,ENST00000610222,ENSG00000134207,protein_coding,1,114636381,114696541,-,ATGAGCGGAGTGTGGGGGGCCGGCGGGCCTCGGTGCCAGGAGGCGC...,1533
72878,ENST00000610247,ENSG00000121716,protein_coding,7,99933737,99965355,+,ATGGGTCGGCCCCTGCTGCTGCCCCTGCTGCTCCTGCTGCAGCCGC...,684


In [7]:
human_transcript_table.chr.value_counts()

1     5574
19    4054
11    3681
2     3629
17    3445
3     3202
12    3202
6     2836
7     2560
16    2486
5     2406
10    2180
X     2169
4     2134
9     2100
8     1990
14    1938
15    1787
20    1428
22    1278
18     897
13     796
21     634
Y      120
MT       7
Name: chr, dtype: int64

In [8]:
def select_regions(row, selected_regions):
    region_start = 0
    region_end = region_start + K
    end = row.length

    while region_end <= end:
       
        region = pd.Series([row.gene_id, row.biotype, row.chr, row.start + region_start, row.start + region_end, row.strand, row.coding_sequence[region_start:region_end]], 
                                    index=["gene_id", "biotype", "chr", "start", "end", "strand", "sequence"])
        selected_regions = selected_regions.append(region, ignore_index=True)
        
        region_start = region_end
        region_end = region_end + K
    
    return selected_regions

In [9]:
selected_regions = pd.DataFrame(columns=["gene_id", "biotype", "chr", "start", "end", "strand", "sequence"])
selected_regions = pd.concat(list(human_transcript_table.apply(lambda x: select_regions(x, selected_regions), axis=1)), ignore_index=True)
selected_regions.head()

,gene_id,biotype,chr,start,end,strand,sequence
0,ENSG00000004059,protein_coding,7,127228399,127228527,+,ATGGGCCTCACCGTGTCCGCGCTCTTTTCGCGGATCTTCGGGAAGA...
1,ENSG00000004059,protein_coding,7,127228527,127228655,+,CACCACCATCCCAACCATAGGCTTCAATGTAGAAACAGTGGAATAT...
2,ENSG00000004059,protein_coding,7,127228655,127228783,+,AGGGCCTCATCTTTGTGGTGGACAGTAATGACCGGGAGCGGGTCCA...
3,ENSG00000004059,protein_coding,7,127228783,127228911,+,GACATGCCCAACGCCATGCCCGTGAGCGAGCTGACTGACAAGCTGG...
4,ENSG00000003056,protein_coding,12,9092961,9093089,-,ATGTTCCCTTTCTACAGCTGCTGGAGGACTGGACTGCTACTACTAC...


In [10]:
human_transcript_table.shape, selected_regions.shape

((56533, 9), (664044, 7))

In [11]:
selected_regions.biotype.value_counts()

protein_coding    664044
Name: biotype, dtype: int64

In [12]:
with open(OUTPUT_FILE_BED, "w") as outfile:
    for i in range(len(selected_regions)):
        line = "chr" + selected_regions["chr"][i] + "\t" + str(selected_regions['start'][i]) + "\t" + str(selected_regions['end'][i]) + "\n"
        outfile.write(line)

In [13]:
with open(OUTPUT_FILE_CSV, "w") as outfile:
    for i in range(len(selected_regions)):
        line = "chr" + selected_regions["chr"][i] + ":" + str(selected_regions['start'][i]) + "-" + str(selected_regions['end'][i]) + "\t" + selected_regions['sequence'][i] +"\n"
        outfile.write(line)

### Keep only genes with at least one binding site of rbp24 dataset

In [42]:
import pybedtools
from pathlib import Path
import os

In [104]:
human_transcript_table_bed = human_transcript_table[["chr", "start", "end", "strand"]]
human_transcript_table_bed["chr"] = "chr" + human_transcript_table["chr"]
human_transcript_table_bed.reset_index(drop=True)

/home/mrkvrbl/miniconda3/envs/diplomka/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,chr,start,end,strand
0,chr7,127228399,127231759,+
1,chr12,9092961,9102551,-
2,chr11,64073050,64084210,+
3,chr12,2904119,2913124,+
4,chr2,72356367,72375167,-
...,...,...,...,...
56528,chr2,27851919,27873713,+
56529,chr14,21051054,21058417,-
56530,chr1,114636381,114696541,-
56531,chr7,99933737,99965355,+


In [44]:
human_transcript_table_bed.to_csv("/home/mrkvrbl/Diplomka/Data/outputs/random_transcripts_rbp24", sep="\t", index=False, header=False)

In [71]:
def intersection(a, b):
    a = pybedtools.BedTool(a)
    b = pybedtools.BedTool(b)

    return a.intersect(b,  u=True, wa=True)

In [72]:
transcript = "/home/mrkvrbl/Diplomka/Data/outputs/random_transcripts_rbp24"
rbp24_path = "/home/mrkvrbl/Diplomka/Data/rbp24/processed"

In [92]:
intersections = []
for root, dirs, files in os.walk(rbp24_path):
    if root.endswith("train"):
        pos_bed_path = root + "/positives.bed"
        inter = intersection(transcript, pos_bed_path)
        inter_df = pd.read_table(inter.fn, names=["chr", "start", "end", "strand"])
        intersections.append(inter_df)


In [103]:
rbp24_transcripts = pd.concat(intersections, names=["chr", "start", "end", "strand"]).drop_duplicates().reset_index(drop=True)
rbp24_transcripts

,chr,start,end,strand
0,chr7,127228399,127231759,+
1,chr12,9092961,9102551,-
2,chr12,2904119,2913124,+
3,chr7,91741465,91763844,-
4,chr1,24742293,24799466,+
...,...,...,...,...
42330,chr6,83903104,83906382,+
42331,chr13,43894987,43935213,-
42332,chr12,94656297,94699374,+
42333,chr14,58030640,58332780,-


In [111]:
rbp24_transcripts.to_csv("/home/mrkvrbl/Diplomka/Data/outputs/transcripts_rbp24.bed", sep="\t", index=False, header=False)

for each protein, pick random coordinates from dataset above, than map them to refence genome.